## Relatório sobre a Implementação do Algoritmo BIKE

### 1. Motivação para Algoritmos de Criptografia Pós-Quântica

A computação quântica representa uma ameaça existencial para muitos dos sistemas criptográficos em uso atualmente. Algoritmos clássicos como o RSA e o Diffie-Hellman, que fundamentam a segurança de comunicações digitais, dependem de problemas matemáticos (fatorização de inteiros grandes e logaritmos discretos, respetivamente) que seriam resolvidos eficientemente por computadores quânticos suficientemente poderosos, como demonstrado pelo algoritmo de Shor.

Esta vulnerabilidade impulsiona a necessidade de desenvolver e padronizar **criptosistemas pós-quânticos (PQC)**, que sejam seguros contra ataques tanto de computadores clássicos quanto quânticos. O objetivo é encontrar problemas que se acreditem ser "difíceis" mesmo para computadores quânticos.

Uma motivação adicional é a necessidade de compactar as chaves criptográficas. Enquanto algumas abordagens seguras, como as baseadas em HLs com matrizes geradas aleatoriamente, podem resultar em chaves de tamanho impraticável (Mbytes).

### 2. O que Muda de um Algoritmo Pós-Quântico para um Clássico

A principal mudança reside nos **fundamentos matemáticos** e nos **problemas de difícil resolução** em que a segurança se baseia:

1.  **Problemas Difíceis Subjacentes:**
    * **Clássicos:** Baseiam-se maioritariamente na dificuldade da fatorização de inteiros grandes (RSA), do logaritmo discreto (DH, ECC) e problemas relacionados em teoria dos números.
    * **Pós-Quânticos:** Utilizam uma gama mais variada de problemas que se acredita serem resistentes a algoritmos quânticos, tais como:
        * Problemas em reticulados (lattices), como o Shortest Vector Problem (SVP) e o Learning With Errors (LWE).
        * Problemas de descodificação em teoria de códigos (como o problema da descodificação de um código linear aleatório).
        * Resolução de sistemas de equações polinomiais multivariadas.
        * Segurança de esquemas de assinatura baseados em hash.

2.  **Estruturas Algébricas:**
    * **Clássicos:** Frequentemente operam sobre grupos finitos, corpos e anéis de inteiros.
    * **Pós-Quânticos:** Introduzem estruturas como reticulados sobre ideais de anéis de polinómios ($\mathbb{Z}[w]$, $\mathbb{Z}_q[w]$) e códigos lineares sobre corpos finitos ($\mathbb{F}_q^n$). O BIKE, por exemplo, usa códigos quasi-cíclicos LDPC (Low-Density Parity-Check) sobre $\mathbb{F}_2$.

3.  **Tamanho das Chaves e Criptogramas:**
    * Muitos algoritmos PQC, especialmente os iniciais baseados em reticulados gerais ou códigos aleatórios, tendem a ter chaves públicas e/ou criptogramas significativamente maiores do que os seus homólogos clássicos para um nível de segurança comparável. Há um esforço contínuo para otimizar esses tamanhos, como visto no BIKE, que usa uma estrutura de código LDPC com matrizes de paridade quasi-cíclicas para compressão.

4.  **Operações Criptográficas:**
    * As operações matemáticas fundamentais mudam. Em vez de exponenciação modular, podem envolver multiplicação de polinómios, operações matriciais sobre corpos finitos, ou algoritmos de descodificação de códigos. O BIKE, por exemplo, utiliza a multiplicação de polinómios no anel $\mathcal{R} = \mathbb{F}_2[w]/(w^\ell + 1)$ e um algoritmo de descodificação chamado Bit-Flip.

5.  **Paradigmas de Segurança:**
    * Enquanto os objetivos (confidencialidade, integridade, autenticidade) permanecem os mesmos, as provas de segurança e os modelos de ataque são adaptados para considerar as capacidades dos adversários quânticos. Muitos PQC são construídos como KEMs (Key Encapsulation Mechanisms) e depois transformados em PKEs (Public Key Encryption schemes) usando transformações como Fujisaki-Okamoto para alcançar segurança CCA2.

### 3. Implementação

O BIKE (Bit Flipping Key Encapsulation) era um dos candidatos baseado em códigos LDPC (Low-Density Parity-Check) quasi-cíclicos. A sua segurança depende da dificuldade de descodificar um código linear a partir de uma matriz geradora pública que não revela a estrutura esparsa da matriz de paridade secreta.

#### Fase 0: Configuração Inicial e Estrutura Algébrica

A base algébrica do BIKE é um anel de polinómios sobre $\mathbb{F}_2$.

* `block_size` ($\ell$): A dimensão do bloco, que é um primo. No caso, $\ell = 257$.
* `error_weight` ($t$): O peso de Hamming (número de coeficientes não-nulos) do vetor de erro. $t = 16$, que será muito menor que o numero de 0s, daí ser low density.
* `total_length` ($n$): O comprimento total do código, que é $n = 2\ell$.
* O anel é $\mathcal{R} = \mathbb{F}_2[w]/(w^\ell + 1)$.

In [ ]:
from random import shuffle
import numpy as np
from cryptography.hazmat.primitives import hashes
from sage.rings.finite_rings.finite_field_constructor import GF
from sage.rings.polynomial.polynomial_ring_constructor import PolynomialRing
from sage.rings.quotient_ring import QuotientRing
from sage.modules.free_module import VectorSpace
from sage.matrix.constructor import Matrix
from sage.matrix.special import block_matrix

# Ring initialization and parameter setup
block_size = 257  # Block dimension
error_weight = 16   # Error vector weight
total_length = 2 * block_size  # Complete vector size
field_base = GF(2)
poly_ring = PolynomialRing(field_base, name='w')
w = poly_ring.gen()
quotient_ring = QuotientRing(poly_ring, poly_ring.ideal(w^block_size + 1))

**Explicação:**
Esta secção define as constantes globais para o algoritmo, como o tamanho do bloco $\ell$ (que corresponde ao grau dos polinómios e à dimensão das matrizes circulantes), o peso dos erros $t$ que o algoritmo pode corrigir, e o comprimento total $n$ dos vetores de código. Estabelece o corpo base $\mathbb{F}_2$ e o anel de polinómios $\mathcal{R} = \mathbb{F}_2[w]/(w^\ell+1)$ onde todas as operações polinomiais (adição, multiplicação) serão realizadas.

#### Fase 1: Geração de Chaves (KeyGen)

A geração de chaves no BIKE envolve a criação de dois polinómios esparsos $h_0, h_1 \in \mathcal{R}$ que formam a chave privada. A chave pública é derivada destes.

* **Chave Privada ($sk$):** Um par de polinómios esparsos $(h_0, h_1)$. A esparsidade é a "trapdoor".
* **Chave Pública ($pk$):** Um par $(g_0, g_1)$ onde,  $g_0 = 1$ e $g_1 = h_0 \cdot h_1^{-1}$. Estes polinómios $g_0, g_1$ são densos e não revelam $h_0, h_1$.

In [ ]:
def generate_sparse_polynomial(density=3):
    """Creates sparse polynomial with 'density' non-zero coefficients."""
    coefficient_list = [1] * density + [0] * (block_size - 2 - density)
    shuffle(coefficient_list)
    return quotient_ring([1] + coefficient_list + [1])

def generate_key_pair():
    """Generates public-private key pair."""
    secret_poly_1 = generate_sparse_polynomial() # h0
    secret_poly_2 = generate_sparse_polynomial() # h1
    private_key = (secret_poly_1, secret_poly_2)
    # pk = (g0, g1) where g0=1, g1 = h0/h1
    public_key = (1, secret_poly_1 / secret_poly_2)
    print(f"Public key generated: {public_key}")
    return private_key, public_key

**Explicação:**
A função `generate_sparse_polynomial` cria um polinómio em $\mathcal{R}$ com um pequeno número de coeficientes não-nulos (esparso), garantindo que o primeiro e o último coeficiente sejam 1. A função `generate_key_pair` chama `generate_sparse_polynomial` duas vezes para obter $h_0$ (`secret_poly_1`) e $h_1$ (`secret_poly_2`), que constituem a chave privada $sk$. A chave pública $pk$ é então calculada como $(1, h_0 \cdot h_1^{-1})$. A inversão de $h_1$ é possível porque $h_1$ é escolhido para ser invertível em $\mathcal{R}$.

#### Fase 2: Encapsulamento de Segredo (Encaps)

O processo de encapsulamento gera uma chave secreta partilhada e o seu respetivo criptograma (encapsulamento) usando a chave pública.

1.  Gerar aleatoriamente dois polinómios de erro "pequenos" (esparsos) $e_0, e_1 \in \mathcal{R}$. O peso total combinado é $t$.
2.  Calcular a chave partilhada $K = \text{Hash}(e_0, e_1)$.
3.  Gerar um polinómio aleatório denso $r \in \mathcal{R}$.
4.  Calcular o criptograma (encapsulamento) $c = (c_0, c_1) = (r \cdot g_0 + e_0, r \cdot g_1 + e_1)$.

In [ ]:
def create_error_vectors(weight):
    """Creates two error polynomials with combined weight."""
    elements = [1] * weight + [0] * (total_length - weight)
    shuffle(elements)
    return quotient_ring(elements[:block_size]), quotient_ring(elements[block_size:])

def compute_hash(first_poly, second_poly):
    """Computes SHA256 hash of two polynomials."""
    hasher = hashes.Hash(hashes.SHA256())
    hasher.update(str(first_poly).encode())
    hasher.update(str(second_poly).encode())
    return hasher.finalize()

def encapsulate_secret(public_key):
    """Encapsulates a shared secret."""
    error_1, error_2 = create_error_vectors(error_weight) # e0, e1
    shared_secret = compute_hash(error_1, error_2)       # K = Hash(e0, e1)
    random_element = quotient_ring.random_element()      # r
    # c0 = r*g0 + e0, c1 = r*g1 + e1
    # g0 is public_key[0] (which is 1), g1 is public_key[1]
    ciphertext = (random_element * public_key[0] + error_1, \
                  random_element * public_key[1] + error_2)
    print(f"Secret encapsulated: {shared_secret.hex()}")
    print(f"Ciphertext created: {ciphertext}")
    return shared_secret, ciphertext

**Explicação:**
`create_error_vectors` gera dois polinómios $e_0$ (`error_1`) e $e_1$ (`error_2`) cuja soma dos pesos de Hamming é igual a `error_weight` ($t$). A chave partilhada (`shared_secret`) é o hash SHA256 destes dois polinómios de erro. Um polinómio aleatório $r$ (`random_element`) é gerado em $\mathcal{R}$. O criptograma $c=(c_0, c_1)$ (`ciphertext`) é então calculado usando a chave pública $(g_0, g_1)$ como $c_0 = r \cdot g_0 + e_0$ e $c_1 = r \cdot g_1 + e_1$.

#### Fase 3: Desencapsulamento de Segredo (Decaps)

O desencapsulamento recupera a chave secreta partilhada $K$ a partir do criptograma $c=(c_0, c_1)$ usando a chave privada $sk=(h_0, h_1)$.

1.  Calcular o **síndroma** $s = c_0 \cdot h_0 + c_1 \cdot h_1$. \
    Como $g_0 \cdot h_0 + g_1 \cdot h_1 = 1 \cdot h_0 + (h_0 \cdot h_1^{-1}) \cdot h_1 = h_0 + h_0 = 0$ em $\mathbb{F}_2[w]/(w^\ell+1)$, então: \
    $s = (r \cdot g_0 + e_0) \cdot h_0 + (r \cdot g_1 + e_1) \cdot h_1$ \
    $s = r \cdot (g_0 \cdot h_0 + g_1 \cdot h_1) + e_0 \cdot h_0 + e_1 \cdot h_1$ \
    $s = e_0 \cdot h_0 + e_1 \cdot h_1$. \
    O síndroma depende apenas dos erros e da chave privada. 
2.  Usar um algoritmo de descodificação (Bit-Flip) com $s$ e a matriz de paridade $H$ (implicitamente definida por $h_0, h_1$) para recuperar $e_0, e_1$.
    A matriz de paridade $H$ (ou sua transposta $H^T$ como usada no código) tem a forma: \
    $H^T = \begin{pmatrix} \mathbf{rot}(h_0) \\ \mathbf{rot}(h_1) \end{pmatrix}$,\
     onde $\mathbf{rot}(h)$ é a matriz circulante gerada pelo polinómio $h$.
3.  Recalcular a chave partilhada $K' = \text{Hash}(e_0, e_1)$. \
    Se $K' = K$, o processo foi bem-sucedido.

In [ ]:
def polynomial_to_vector_r(polynomial):
    """Converts polynomial to vector of size block_size."""
    vector_space = VectorSpace(field_base, block_size)
    return vector_space(polynomial.list() + [0] * (block_size - len(polynomial.list())))

def polynomial_pair_to_vector_n(cipher_pair):
    """Converts polynomial pair to vector of size total_length."""
    vector_space = VectorSpace(field_base, total_length)
    combined_list = polynomial_to_vector_r(cipher_pair[0]).list() + \
                    polynomial_to_vector_r(cipher_pair[1]).list()
    return vector_space(combined_list)

def rotate_vector(input_vector):
    """Rotates vector by one position."""
    vector_space = VectorSpace(field_base, block_size)
    output_vector = vector_space()
    output_vector[0] = input_vector[-1]
    for idx in range(block_size - 1):
        output_vector[idx + 1] = input_vector[idx]
    return output_vector

def build_rotation_matrix(polynomial):
    """Builds rotation matrix from polynomial."""
    matrix = Matrix(field_base, block_size, block_size)
    matrix[0] = polynomial_to_vector_r(polynomial)
    for row_idx in range(1, block_size):
        matrix[row_idx] = rotate_vector(matrix[row_idx - 1])
    return matrix

def calculate_hamming_weight(vector):
    """Computes Hamming weight of vector."""
    return sum([1 if element == 1 else 0 for element in vector])

def iterative_bit_correction(parity_matrix_H, received_vector, syndrome):
    """Bit-flipping algorithm for error correction."""
    current_guess_e = vector(field_base, total_length, [0]*total_length) # Initial guess for error e = (e0,e1)
    current_syndrome = syndrome # This is s = e*H^T
    
    max_iterations = block_size # A common heuristic for max iterations
    
    # H_columns will store the columns of H
    H_columns = [parity_matrix_H.column(j) for j in range(total_length)]

    iter_count = 0
    while calculate_hamming_weight(current_syndrome) > 0 and iter_count < max_iterations:
        iter_count += 1
        
        # error_weights[j] = |current_syndrome INTERSECT j-th column of H|
        error_weights = [calculate_hamming_weight(current_syndrome * H_columns[j]) for j in range(total_length)]
        
        peak_weight = max(error_weights) if error_weights else 0
        
        if peak_weight == 0: # No improvement possible or syndrome is already zero
            break
            
        flipped_one_bit_in_iter = False
        for position in range(total_length):
            if error_weights[position] == peak_weight:
                current_guess_e[position] = 1 - current_guess_e[position] # Bit flip on error guess e
                current_syndrome += H_columns[position].row() # Add the row vector
                flipped_one_bit_in_iter = True 
        
        if not flipped_one_bit_in_iter and peak_weight > 0: 
            pass # For robustness

    if iter_count == max_iterations and calculate_hamming_weight(current_syndrome) > 0:
        print(f"Bit-Correction: Maximum iterations ({max_iterations}) reached, syndrome weight: {calculate_hamming_weight(current_syndrome)}")
        return None # Failed to correct
    
    if calculate_hamming_weight(current_syndrome) > 0:
        print(f"Bit-Correction: Failed to decode, syndrome weight: {calculate_hamming_weight(current_syndrome)}")
        return None # Failed to correct
        
    # current_guess_e is the recovered error vector e = (e0, e1)
    return current_guess_e

def decapsulate_secret(private_key, ciphertext):
    """Decapsulates the shared secret."""
    h0 = private_key[0]
    h1 = private_key[1]

    rot_h0 = build_rotation_matrix(h0)
    rot_h1 = build_rotation_matrix(h1)

    # H^T is (2l x l). H^T = [ rot(h0)^T ]
    #                          [ rot(h1)^T ]
    H_transposed_sage = block_matrix(2, 1, [rot_h0, rot_h1])

    cipher_vector = polynomial_pair_to_vector_n(ciphertext) # c = (c0,c1) as 1x2l vector

    # syndrome_vector = c_vec * H_transposed_sage ( (1x2l) * (2lxl) = 1xl )
    syndrome_vector = cipher_vector * H_transposed_sage

    # For BitFlip, we need H itself. H = [rot_h0 | rot_h1]
    H_for_bitflip = block_matrix(1, 2, [rot_h0, rot_h1]) # l x 2l matrix

    recovered_error_vector = iterative_bit_correction(H_for_bitflip, cipher_vector, syndrome_vector)

    if recovered_error_vector is None:
        print("Decapsulate: Bit-correction failed")
        return None

    error_list = recovered_error_vector.list()
    recovered_error_1 = quotient_ring(error_list[:block_size]) # e0
    recovered_error_2 = quotient_ring(error_list[block_size:]) # e1

    # Verification: Check if the recovered error's weight is correct.
    current_error_weight = calculate_hamming_weight(polynomial_to_vector_r(recovered_error_1)) + \
                           calculate_hamming_weight(polynomial_to_vector_r(recovered_error_2))
    if current_error_weight != error_weight:
        print(f"Decapsulate: Recovered error weight {current_error_weight} does not match expected {error_weight}")

    # Verification using the original formulation of syndrome s = e0*h0 + e1*h1
    s_check_poly = recovered_error_1 * h0 + recovered_error_2 * h1
    s_check_vec = polynomial_to_vector_r(s_check_poly)
    if s_check_vec != syndrome_vector:
        print("Decapsulate: Syndrome check failed for recovered error.")

    recovered_secret = compute_hash(recovered_error_1, recovered_error_2)
    print(f"Secret decapsulated: {recovered_secret.hex()}")
    return recovered_secret

**Explicação:**
1. Primeiro, as funções auxiliares convertem polinómios em vetores e constroem matrizes circulantes (`build_rotation_matrix`) a partir de $h_0$ e $h_1$.\
Estas matrizes são combinadas para formar $H$ e $H^T$. 
2. O criptograma $c=(c_0, c_1)$ (`ciphertext`) é convertido para um vetor (`cipher_vector`). \
O síndroma $s$ (`syndrome_vector`) é calculado como $s = c_0 \cdot h_0 + c_1 \cdot h_1$ (realizado por multiplicação matricial `cipher_vector * H_transposed_sage`). \
3. A função `iterative_bit_correction` (algoritmo Bit-Flip) é então chamada. Ela recebe $H$ (`H_for_bitflip`), o `cipher_vector` (que não é usado diretamente como palpite inicial no Bit-Flip padrão, que geralmente tenta encontrar $e$ do zero ou de $c H^T$), e o `syndrome_vector` ($s = eH^T$).
O Bit-Flip tenta iterativamente encontrar o erro $e=(e_0, e_1)$ tal que $s = e_0 h_0 + e_1 h_1$. A cada iteração, ele calcula, para cada posição de bit $j$ do erro $e$, o número de equações de paridade não satisfeitas em que o bit $e_j$ participa ($|s \cap H_{col_j}|$, onde $H_{col_j}$ é a $j$-ésima coluna de $H$). Os bits de $e$ correspondentes ao maior valor são invertidos ("flipped"), e o síndroma é atualizado adicionando a coluna de $H$ correspondente.

4. Se bem-sucedido, o algoritmo retorna o vetor de erro recuperado $e = (e_0, e_1)$ (`recovered_error_vector`). Finalmente, a chave secreta partilhada (`recovered_secret`) é recalculada usando o hash de $(e_0, e_1)$ recuperados.


### 4. Transformação de Fujisaki-Okamoto (FO)

O BIKE, como implementado acima, é um **KEM-CPA** (Key Encapsulation Mechanism com segurança contra ataques de texto cifrado escolhido). No entanto, para aplicações práticas, frequentemente necessitamos de segurança **CCA2** (Chosen Ciphertext Attack 2), que é mais robusta contra adversários que podem realizar consultas de decifração.

A **Transformação de Fujisaki-Okamoto (FO)** é uma técnica criptográfica que permite converter um KEM-CPA em um **PKE-CCA2** (Public Key Encryption com segurança CCA2). Esta transformação é amplamente utilizada em criptografia pós-quântica.

#### Princípios da Transformação FO

1. **Aleatoriedade Determinística**: Em vez de usar aleatoriedade pura, a FO usa uma função hash para gerar a aleatoriedade a partir da mensagem.
2. **Verificação de Consistência**: Durante a decifração, o algoritmo re-encripta a mensagem recuperada e verifica se o resultado coincide com o criptograma original.
3. **Segurança CCA2**: Estas verificações garantem que qualquer modificação do criptograma seja detectada, proporcionando segurança contra ataques de texto cifrado escolhido.

#### Estrutura da Transformação

A transformação FO converte o KEM BIKE num esquema PKE através dos seguintes passos:

**cifra PKE:**
1. Para cifrar uma mensagem $m$, escolher aleatoriedade $r$
2. Calcular $m' = m \oplus G(r)$ onde $G$ é uma função hash
3. Usar o KEM para encapsular $r$ obtendo $(K, c_1)$
4. Calcular $c_2 = m' \oplus H(K)$ onde $H$ é outra função hash
5. O criptograma final é $(c_1, c_2)$

**decifração PKE:**
1. Usar o KEM para desencapsular $c_1$ obtendo $K'$
2. Calcular $m' = c_2 \oplus H(K')$
3. Recuperar $r'$ através da relação $m' = m \oplus G(r')$
4. **Verificação**: Re-encapsular $r'$ e verificar se o resultado coincide com $c_1$
5. Se a verificação passa, retornar $m$; caso contrário, retornar erro

#### Implementação no BIKE

A implementação fornecida inclui as seguintes funções para a transformação FO:

In [ ]:
# Implementação da Transformação de Fujisaki-Okamoto

def hash_function_g(input_element):
    """Hash function g for transformation."""
    hasher = hashes.Hash(hashes.SHA256())
    hasher.update(str(input_element).encode())
    return hasher.finalize()

def xor_operation(data_bytes, mask_bytes):
    """XOR operation with mask repetition."""
    output = b''
    data_len = len(data_bytes)
    mask_len = len(mask_bytes)
    byte_index = 0
    while byte_index < data_len:
        for mask_index in range(mask_len):
            if byte_index < data_len:
                output += (data_bytes[byte_index] ^^ mask_bytes[mask_index]).to_bytes(1, byteorder='big')
                byte_index += 1
            else:
                break
    return output

def encryption_function_f(public_key, message_poly, error_1, error_2):
    """Function f for PKE scheme."""
    ciphertext_pair = (message_poly * public_key[0] + error_1, message_poly * public_key[1] + error_2)
    derived_key = compute_hash(error_1, error_2)
    return derived_key, ciphertext_pair

def extract_errors_from_ciphertext(private_key, error_ciphertext):
    """Extracts errors from ciphertext."""
    rotation_matrix_1 = build_rotation_matrix(private_key[0])
    rotation_matrix_2 = build_rotation_matrix(private_key[1])
    parity_check = block_matrix(2, 1, [rotation_matrix_1, rotation_matrix_2])
    error_vector = polynomial_pair_to_vector_n(error_ciphertext)
    syndrome_vector = error_vector * parity_check
    corrected_error = iterative_bit_correction(parity_check, error_vector, syndrome_vector)
    if corrected_error is None:
        print("ExtractErrors: Iteration limit reached")
        return None
    error_list = corrected_error.list()
    recovered_error_1 = quotient_ring(error_list[:block_size])
    recovered_error_2 = quotient_ring(error_list[block_size:])
    original_error_1 = error_ciphertext[0] - recovered_error_1
    original_error_2 = error_ciphertext[1] - recovered_error_2
    return original_error_1, original_error_2

def derive_key_from_errors(error_1, error_2):
    """Derives key from error polynomials."""
    if calculate_hamming_weight(polynomial_to_vector_r(error_1)) + calculate_hamming_weight(polynomial_to_vector_r(error_2)) != error_weight:
        print("DeriveKey: Decoding error detected")
        return None
    derived_key = compute_hash(error_1, error_2)
    return derived_key

def pke_generate_keys():
    """Generates key pair for PKE scheme."""
    return generate_key_pair()

def pke_encrypt_message(plaintext_msg, public_key):
    """Encrypts message using FOT."""
    error_1, error_2 = create_error_vectors(error_weight)
    random_poly = quotient_ring.random_element()
    hash_output = hash_function_g(random_poly)
    masked_message = xor_operation(plaintext_msg.encode(), hash_output)
    binary_representation = bin(int.from_bytes(masked_message, byteorder='big'))
    combined_polynomial = quotient_ring(binary_representation)
    encryption_key, error_ciphertext = encryption_function_f(public_key, combined_polynomial + random_poly, error_1, error_2)
    final_ciphertext = xor_operation(str(random_poly).encode(), encryption_key)
    print(f"Message encrypted: masked={masked_message.hex()}, errors={error_ciphertext}, cipher={final_ciphertext.hex()}")
    return masked_message, error_ciphertext, final_ciphertext

def pke_decrypt_message(private_key, masked_msg, error_cipher, final_cipher):
    """Decrypts message using FOT."""
    extracted_error_1, extracted_error_2 = extract_errors_from_ciphertext(private_key, error_cipher)
    if extracted_error_1 is None:
        print("PKE_Decrypt: Failed to extract errors")
        return None
    decryption_key = derive_key_from_errors(extracted_error_1, extracted_error_2)
    if decryption_key is None:
        print("PKE_Decrypt: Failed to derive key")
        return None
    random_xor = xor_operation(final_cipher, decryption_key)
    try:
        recovered_random = quotient_ring(random_xor.decode())
    except:
        print("PKE_Decrypt: Error decoding random element")
        return None
    binary_representation = bin(int.from_bytes(masked_msg, byteorder='big'))
    combined_polynomial = quotient_ring(binary_representation)
    if (decryption_key, error_cipher) != encryption_function_f(public_key, combined_polynomial + recovered_random, extracted_error_1, extracted_error_2):
        print("PKE_Decrypt: Decoding verification failed")
        return None
    hash_output = hash_function_g(recovered_random)
    recovered_plaintext = xor_operation(masked_msg, hash_output)
    print(f"Message decrypted: {recovered_plaintext.decode()}")
    return recovered_plaintext.decode()

**Explicação da Implementação:**

1. **`hash_function_g`**: Implementa a função hash $G$ utilizada para mascarar a mensagem.

2. **`xor_operation`**: Realiza a operação XOR entre dados e uma máscara, repetindo a máscara conforme necessário.

3. **`encryption_function_f`**: Função principal de cifra que:
   - Combina a mensagem mascarada com a aleatoriedade
   - Usa o KEM BIKE para encapsular
   - Retorna a chave derivada e o criptograma

4. **`pke_encrypt_message`**: Implementa o algoritmo completo de cifrar PKE:
   - Gera aleatoriedade $r$
   - Mascara a mensagem: $m' = m \oplus G(r)$
   - Encapsula usando o KEM
   - Produz o criptograma final

5. **`pke_decrypt_message`**: Implementa o algoritmo de decifração PKE:
   - Extrai os erros do criptograma
   - Deriva a chave de decifração
   - Recupera a aleatoriedade
   - **Verifica a consistência** re-encriptando
   - Retorna a mensagem original ou erro

In [ ]:
if __name__ == "__main__":
    print("Iniciando teste KEM...")
    # Geração de Chaves
    sk, pk = generate_key_pair()
    print(f"Chave Privada (h0, h1): ({sk[0]}, {sk[1]})")
    print(f"Chave Pública (g0, g1): ({pk[0]}, {pk[1]})")
    
    # Encapsulamento
    k_original, c = encapsulate_secret(pk)
    print(f"Segredo Original K: {k_original.hex()}")
    print(f"Criptograma c=(c0,c1): ({c[0]}, {c[1]})")
    
    # Desencapsulamento
    k_recuperado = decapsulate_secret(sk, c)
    
    if k_recuperado:
        print(f"Segredo Recuperado K': {k_recuperado.hex()}")
        if k_original == k_recuperado:
            print("SUCESSO: O segredo original e o recuperado coincidem.")
        else:
            print("FALHA: O segredo original e o recuperado NÃO coincidem.")
    else:
        print("FALHA: O desencapsulamento não produziu um segredo.")

    print("\n" + "="*50)
    print("TESTE DO PKE COM TRANSFORMAÇÃO FUJISAKI-OKAMOTO")
    print("="*50)

    # Teste da implementação PKE
    test_message = "Mensagem secreta para cifrar!"
    print(f"Mensagem Original: {test_message}")

    # Geração de chaves para PKE
    sk_pke, pk_pke = pke_generate_keys()
    print(f"Chaves PKE geradas.")

    # Cifra PKE
    masked_msg, error_cipher, final_cipher = pke_encrypt_message(test_message, pk_pke)
    print(f"Mensagem encriptada com sucesso.")

    # decifração PKE
    recovered_message = pke_decrypt_message(sk_pke, masked_msg, error_cipher, final_cipher)

    if recovered_message:
        print(f"Mensagem Recuperada: {recovered_message}")
        if test_message == recovered_message:
            print("SUCESSO: A mensagem original e a recuperada coincidem.")
        else:
            print("FALHA: A mensagem original e a recuperada NÃO coincidem.")
    else:
        print("FALHA: A decifração PKE não produziu uma mensagem.")

    print("\n" + "="*50)
    print("RESUMO DOS TESTES")
    print("="*50)
    print(f"KEM BIKE: {'PASSOU' if k_original == k_recuperado else 'FALHOU'}")
    print(f"PKE com FO: {'PASSOU' if test_message == recovered_message else 'FALHOU'}")
    print("="*50)